# Model Training

### Imports

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import boto3
import re
import os
import io
import time
import tarfile
import random
import string
import matplotlib.pyplot as plt
from io import BytesIO
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.cluster import KMeans
import sagemaker
from sagemaker import PCA, KMeans, get_execution_role, Session
from sagemaker import KMeansModel, KMeansModel
from sklearn.metrics import silhouette_score
from sagemaker.model import Model
from sagemaker.predictor import Predictor
import json
from sagemaker.serializers import JSONSerializer
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer
import pickle

## Load Training Dataset from S3

In [7]:
train_df = pd.read_csv('s3://team6datasets/splitDatasets/train.csv')

In [8]:
train_df.shape

(284778, 46)

In [9]:
train_df.head()

,CustomerID,gender_Agender,gender_Bigender,gender_Female,gender_Genderfluid,gender_Genderqueer,gender_Male,gender_Non-binary,gender_Polygender,Education_2n Cycle,...,SpendingCategory_Medium,SpendingCategory_High,SpendingCategory_Very High,IncomeLevel_Low,IncomeLevel_Medium,IncomeLevel_High,IncomeLevel_Very High,Purchase_Frequency,Age,Total_Spent_byCustomer
0,4426,0,0,0,0,0,1,0,0,1,...,1,0,0,0,0,0,1,0.652632,0.892857,0.021736
1,1100,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,1,0.589474,0.839286,0.027672
2,3602,0,0,1,0,0,0,0,0,1,...,1,0,0,0,1,0,0,0.547368,0.678571,0.021624
3,5268,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,1,0.421053,0.714286,0.028717
4,716,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,1,0.421053,0.357143,0.024864


## Sagemaker Session and Role

In [10]:
# Define the SageMaker session and role
sagemaker_session = Session()
role = get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


## Dimensionality Reduction using Variance

In [11]:
# Set the variance threshold
variance_threshold = 0.2

# Create a VarianceThreshold object
selector = VarianceThreshold(threshold=variance_threshold)

# Fit the selector to the original data (train_df)
selector.fit(train_df)

# Get the support mask (boolean array indicating features to keep)
support = selector.get_support(indices=True)

# Select features based on the support mask
reduced_features = train_df.iloc[:, support]  # Using .iloc for clarity

# Now 'reduced_features' is a DataFrame containing features with variance exceeding the threshold
print(f"Number of features removed: {len(train_df.columns) - len(reduced_features.columns)}")

Number of features removed: 35


In [12]:
# Show the reduced_features DataFrame
print("Reduced Features DataFrame:")
reduced_features.head()

Reduced Features DataFrame:


,CustomerID,gender_Female,gender_Male,Kidhome_0,Kidhome_1,Kidhome_2,Teenhome_0,Teenhome_1,Teenhome_2,region_Asia,region_Europe
0,4426,0,1,1,0,0,0,1,0,1,0
1,1100,0,1,0,0,1,1,0,0,1,0
2,3602,1,0,0,0,1,0,0,1,1,0
3,5268,0,1,0,1,0,0,1,0,1,0
4,716,0,1,1,0,0,0,1,0,0,1


In [13]:
reduced_features.shape

(284778, 11)

## Model Training with KMeans Algorithm

### Use Elbow Method to Determine Ideal K

In [26]:
import os

# Check if the image file already exists
if os.path.isfile('elbow_method.png'):
    print("Elbow method already run. Image found in folder.")
else:
    # Define a range of k values
    k_values = range(2, 11)  

    # Convert the reduced_features DataFrame to float32
    reduced_features_float32 = reduced_features.astype('float32')

    # Train KMeans models for each k value and compute WCSS
    wcss_values = []
    for k in k_values:
        kmeans = KMeans(role=role,
                        instance_count=1,
                        instance_type='ml.c4.xlarge',
                        k=k)
        kmeans.fit(kmeans.record_set(reduced_features_float32.values)) 
        # Filter the DataFrame for the row where metric_name is 'train:msd'
        wcss_row = kmeans.training_job_analytics.dataframe().loc[kmeans.training_job_analytics.dataframe()['metric_name'] == 'train:msd']
        # Extract the WCSS value from the 'value' column of the filtered row
        wcss_value = wcss_row['value'].iloc[0]
        # Append the WCSS value to the list
        wcss_values.append(wcss_value)

    # Plot the WCSS values against k
    plt.plot(k_values, wcss_values, marker='o')
    plt.title('Elbow Method')
    plt.xlabel('Number of clusters (k)')
    plt.ylabel('Within-cluster sum of squares (WCSS)')
    plt.savefig('elbow_method.png')  # Save the plot as an image
    plt.show()
    print("Elbow method run successfully. Image saved in folder.")


Elbow method already run. Image found in folder.


In [14]:
# Define a function to generate a random 3-digit serial number
def generate_serial_number():
    return ''.join(random.choices(string.digits, k=3))

# Drop the CustomerID column from the feature DataFrame
features_for_training = reduced_features.drop(columns=['CustomerID'])

# Store CustomerID separately for reference
customer_ids = reduced_features['CustomerID']

# Initialize S3 client
s3 = boto3.client('s3')

# Define hyperparameters for KMeans model
hyperparameters = {'k': 4, 'init_method': 'random'}

# Set the output path
output_path = 's3://team6datasets/kmeans_model_output/'

# Set k value
k_value = hyperparameters['k']
model_output_path = f'{output_path}kmeans-{k_value}/output/'

# Train the model
print(f"Training model for k={k_value}...")

# Convert DataFrame to NumPy array (excluding CustomerID)
features_np = features_for_training.values.astype('float32')

# Create KMeans estimator
kmeans = KMeans(role=role,
                instance_count=1,
                instance_type='ml.c4.xlarge',
                k=k_value,
                output_path=output_path)

# Generate a random 3-digit serial number for the job name
serial_number = generate_serial_number()
# Append the serial number to the job name
job_name = f"kmeans-{k_value}-{serial_number}"


# Set the output path with job name
model_output_path = f'{output_path}/{job_name}/output/'

# Fit the model with the updated output path
kmeans.fit(kmeans.record_set(features_np), job_name=job_name)

print(f"Model for k={k_value} trained and saved to {model_output_path}")


Training model for k=4...
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: kmeans-4-142


2024-04-09 03:49:15 Starting - Starting the training job...
2024-04-09 03:49:31 Starting - Preparing the instances for training...
2024-04-09 03:50:05 Downloading - Downloading input data...
2024-04-09 03:50:40 Downloading - Downloading the training image.........
2024-04-09 03:52:11 Training - Training image download completed. Training in progress.
2024-04-09 03:52:11 Uploading - Uploading generated training modelDocker entrypoint called with argument(s): train
Running default environment configuration script
[04/09/2024 03:52:05 INFO 140689755305792] Reading default configuration from /opt/amazon/lib/python3.8/site-packages/algorithm/resources/default-input.json: {'init_method': 'random', 'mini_batch_size': '5000', 'epochs': '1', 'extra_center_factor': 'auto', 'local_lloyd_max_iter': '300', 'local_lloyd_tol': '0.0001', 'local_lloyd_init_method': 'kmeans++', 'local_lloyd_num_trials': 'auto', 'half_life_time_size': '0', 'eval_metrics': '["msd"]', 'force_dense': 'true', '_disable_wait_

## Model Validation

### Load Validation Data from S3

In [15]:
validate_df = pd.read_csv('s3://team6datasets/splitDatasets/validation.csv')

In [16]:
validate_df.shape

(61024, 46)

### Dimensionality Reduction using Variance 

In [17]:
# Set the variance threshold
variance_threshold = 0.2

# Create a VarianceThreshold object
selector = VarianceThreshold(threshold=variance_threshold)

# Fit the selector to the original data (train_df)
selector.fit(validate_df)

# Get the support mask (boolean array indicating features to keep)
support = selector.get_support(indices=True)

# Select features based on the support mask
reduced_features_validate = validate_df.iloc[:, support]  # Using .iloc for clarity

# Now 'reduced_features' is a DataFrame containing features with variance exceeding the threshold
print(f"Number of features removed: {len(validate_df.columns) - len(reduced_features_validate.columns)}")

Number of features removed: 35


In [18]:
# Show the reduced_features_validate DataFrame
print("Reduced Features DataFrame:")
reduced_features_validate.head()

Reduced Features DataFrame:


,CustomerID,gender_Female,gender_Male,Kidhome_0,Kidhome_1,Kidhome_2,Teenhome_0,Teenhome_1,Teenhome_2,region_Asia,region_Europe
0,9706,0,0,1,0,0,1,0,0,0,0
1,5555,1,0,0,0,1,0,0,1,0,0
2,3766,0,1,1,0,0,0,0,1,0,1
3,6515,1,0,1,0,0,0,0,1,0,0
4,1158,0,0,0,0,1,1,0,0,0,1


### Validate Model

In [27]:
# Generate a random 3-digit serial number
serial_number = ''.join(random.choices(string.digits, k=3))

# Define the endpoint name with the serial number appended
endpoint_name = f'kmeans-endpoint-{serial_number}'

# Define the role
role = get_execution_role()

# Load the trained model
model_key = f'kmeans_model_output/{job_name}/output/model.tar.gz'
model_location = f's3://team6datasets/{model_key}'

# Create a SageMaker Model object
model = Model(model_data=model_location,
              image_uri='382416733822.dkr.ecr.us-east-1.amazonaws.com/kmeans:1',  
              role=role)

try:
    # Deploy the model to create an endpoint
    predictor = model.deploy(initial_instance_count=1,
                             instance_type='ml.c4.xlarge',
                             endpoint_name=endpoint_name)
    
    print("Endpoint deployed successfully!")

    # Check if predictor is properly initialized
    if predictor is not None and predictor.endpoint_name is not None:
        # Extract features for validation (excluding CustomerID)
        features_for_validation = reduced_features_validate.drop(columns=['CustomerID'])
        features_np_validation = features_for_validation.values.astype('float32')

        # Convert data to JSON format
        json_data = json.dumps(features_np_validation.tolist())  # Convert NumPy array to a list for JSON

        # Serialize the JSON data to bytes
        buffer = BytesIO()
        buffer.write(json_data.encode('utf-8'))  # Encode JSON string as UTF-8 bytes
        buffer.seek(0)
        data_bytes = buffer.read()

        # Predict clusters for the validation data with content type set to application/json
        initial_args = {'ContentType': 'application/json'}
        clusters = predictor.predict(data_bytes, initial_args=initial_args)

        # Add the cluster labels to the validation DataFrame
        reduced_features_validate['Cluster'] = clusters
        print("Prediction successful. Cluster labels added to validation DataFrame.")
    else:
        print("Failed to deploy endpoint or predictor is not properly initialized.")

except Exception as e:
    print("Failed to deploy endpoint:", e)
    
    # Optional: Rollback the endpoint creation if it fails
    if predictor:
        predictor.delete_endpoint()


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


INFO:sagemaker:Creating model with name: kmeans-2024-04-09-05-42-17-820
INFO:sagemaker:Creating endpoint-config with name kmeans-endpoint-095
INFO:sagemaker:Creating endpoint with name kmeans-endpoint-095


------!Endpoint deployed successfully!
Failed to deploy endpoint or predictor is not properly initialized.


## Evaluating KMeans Model